In [0]:
import os
from getpass import getpass
user = getpass('GitHub user')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password
!git clone https://$GITHUB_AUTH@github.com/chengmar09/comp0053.git

GitHub user··········
GitHub password··········
Cloning into 'comp0053'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 33 (delta 11), reused 24 (delta 7), pack-reused 0
Unpacking objects: 100% (33/33), done.


In [0]:
!echo 'Check Current Dir'
%cd /content/comp0053/
!pwd
!ls
!pip install -r Requirement.txt

Check Current Dir
/content/comp0053
/content/comp0053
COMP0053_EmoPain_Demo.ipynb  helper.py	Requirement.txt
data			     README.md	train.py
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=fbff6a177236bfa0224f2d2f9113a91b6a16c50ceb7ef6719849d6be59d22d40
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [0]:
from helper import *

movementDataLoader = DataReader('Movementdata')
#train_data, valid_data = movementDataLoader.get_data()
(train_x, train_y), (valid_x, valid_y) = movementDataLoader.more_more_processing()

Using TensorFlow backend.


Create a ->Movementdata<- folder 

Now downloading training data

Now downloading Validation data
Unzipping training data
Unzipping validation data
There is total of 23 training data
There is total of 12 Validation data


In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Input, GRU
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(Dropout(0.5))
#model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
#model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', f1_m])

# checkpoint
checkpoint_path = "stack_bilstm.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)
callbacks_list = [checkpoint]

history = model.fit(train_x, train_y, validation_split=0.1, epochs=1, callbacks=callbacks_list)  # starts training






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 320496 samples, validate on 35611 samples
Epoch 1/1





320496/320496 [==============================] - 444s 1ms/step - loss: 0.4025 - acc: 0.8377 - f1_m: 0.2376 - val_loss: 0.7060 - val_acc: 0.6967 - val_f1_m: 0.0821

Epoch 00001: saving model to stack_bilstm.ckpt


In [0]:
model.load_weights(checkpoint_path)
validation_score = model.evaluate(valid_x, valid_y)

for name, score in zip(model.metrics_names, validation_score):
    print(f'{name} : {score}')

158438/158438 [==============================] - 41s 260us/step
loss : 0.4764943066514633
acc : 0.8419507946326008
f1_m : 0.013751586921333726
